In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

from activations import Cosine
from networks import WHVIRegression
from layers import WHVILinear
from torch_datasets import ToyDataset

In [2]:
torch.manual_seed(0)  # Seed for reproducibility
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')

# Data
dataset = ToyDataset(n=128, device=device)
data_loader = DataLoader(dataset, batch_size=64)

In [3]:
net = WHVIRegression([
    nn.Linear(1, 128),
    Cosine(),
    WHVILinear(128, lambda_=1e-3, device=device),
    Cosine(),
    WHVILinear(128, lambda_=1e-3, device=device),
    Cosine(),
    nn.Linear(128, 1)
])
net = net.to(device)

gamma=0.0005
p = 0.3
optimizer = optim.Adam(net.parameters(), lr=1e-3)
lr_scheduler = optim.lr_scheduler.LambdaLR(optimizer, lambda t: (1 + gamma * t)**(-p))

In [4]:
# TODO why is MNLL not changing in the fixed variance version???
net.train_model(data_loader, optimizer, epochs1=1500, epochs2=2000)

[Opt. var.] KL = -0.00, MNLL = 33.08:  44%|████▍     | 880/2000 [00:18<00:23, 46.96it/s]        


KeyboardInterrupt: 

In [ ]:
net.eval()
net.eval_samples = 500
x_test = torch.reshape(torch.linspace(-2, 3, 1000), (-1, 1))
y_test = dataset.f(x_test)
y_pred = net(x_test.to(device)).cpu()

In [ ]:
plt.figure()
plt.ylim(-1, 2.5)
plt.xlim(-2, 3)
for i in range(y_pred.size()[2]):
    plt.plot(
        x_test.cpu().numpy().ravel(),
        y_pred[..., i].detach().cpu().numpy().ravel(),
        c='r', 
        alpha=0.05
    )

plt.scatter(dataset.x.cpu(), dataset.y.cpu(), ec='k', label='Noisy training measurements')
plt.plot(x_test, y_test, label='True function')
plt.legend()
plt.show()